Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
library(grid)
library(gridExtra)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
rm(data_bc_clean)
fig_out_path = paste0(here(), '/outputs/fig/')

# Value difference distributions

In [ ]:
normMax = 1
normMin = -1

data_yn_clean = data_yn_clean %>%
  filter(reference != -99) %>%
  filter(rt > .3 & rt < 5) %>% # discard very long and short RT trials
  # filter((type == stim_type) & (subnum == sub_num)) %>%
  group_by(day, subnum, type) %>% # Make sure val diff ranges from -1 to 1 for each day (in this condition and for this subnum)
  mutate(rawVDiff = possiblePayoff - reference,
         normVDiff =  (normMax - normMin) / (max(rawVDiff) - min(rawVDiff)) * (rawVDiff - max(rawVDiff)) + (normMax),
         possiblePayoff_std = possiblePayoff - mean(possiblePayoff),
         oldVDiff = possiblePayoff_std - reference) %>%
  mutate(rtPN = ifelse(yesChosen == 1, rt, (-1)*rt))

In [ ]:
summary(data_yn_clean$normVDiff)

In [ ]:
summary(data_yn_clean$oldVDiff)

In [ ]:
data_yn_clean %>% 
  mutate(type = ifelse(type == 1, "HT", "RE")) %>%
  ggplot(aes(normVDiff, oldVDiff, color = type))+
  geom_point(alpha = .5)+
  facet_wrap(~subnum)+
  scale_color_brewer(palette = "Dark2")+
  theme(legend.position = "bottom")

In [ ]:
data_yn_clean %>%
  ungroup() %>%
  select(subnum, normVDiff, oldVDiff, type) %>%
  gather(key, value, -subnum, -type) %>%
  mutate(type = ifelse(type == 1, "HT", "RE")) %>%
  filter(key == "normVDiff") %>%
  ggplot(aes(value, fill = type))+
  geom_histogram(bins = 30, alpha = .5, position = "identity")+
  facet_grid(key ~ subnum, scales = "free")+
  theme(legend.position = "bottom")+
  labs(y = "", x = "", fill = "")+
  scale_fill_brewer(palette = "Dark2")

In [ ]:
data_yn_clean %>%
  ungroup() %>%
  select(subnum, normVDiff, oldVDiff, type) %>%
  gather(key, value, -subnum, -type) %>%
  mutate(type = ifelse(type == 1, "HT", "RE")) %>%
  filter(key == "oldVDiff") %>%
  ggplot(aes(value, fill = type))+
  geom_histogram(bins = 30, alpha = .5, position = "identity")+
  facet_grid(key ~ subnum, scales = "free")+
  theme(legend.position = "bottom")+
  labs(y = "", x = "", fill = "")+
  scale_fill_brewer(palette = "Dark2")

# Hierarchy over days

## Estimates using normalized VDiff

In [ ]:
stim_types = c("HT", "RE")
subnums = c(601, 609, 611, 619, 621, 629)

in_path = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/hddm/jags_out'

yn_ddm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(subnums)){
    
    cur_stim_type = stim_types[i]
    cur_sub = subnums[j]
    # summary_YN_HDDM_FIT_RE_sub-611.csv
    
    fn = file.path(in_path, paste0('summary_YN_HDDM_FIT_', cur_stim_type,'_sub-', cur_sub,'.csv'))
    
    if(file.exists(fn)){
      cur_summary = read.csv(fn)
      cur_summary = cur_summary %>%
        rename(par = X) %>%
        mutate(stim_type = cur_stim_type,
               subnum = cur_sub)
      
      yn_ddm_summary = rbind(yn_ddm_summary, cur_summary)
    } else {
      next
    }
  }
}
rm(cur_summary)

In [ ]:
yn_ddm_summary

In [ ]:
yn_ddm_summary = yn_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, subnum, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_group = gsub("p\\[", "", par_group),
         par_group = gsub("\\]", "", par_group)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group != "kappa" & par_group != "pr") %>%
  filter(par_group != "mu") %>% #Not showing group estimates in this plot
  rename(day = par_group) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", ifelse(par_name == "b", "d", ifelse(par_name == "alpha", "sigma", par_name)))) %>%
  mutate(par_name = factor(par_name, levels = c("d", "sigma", "bias", "ndt")),
         day = as.numeric(day))

In [ ]:
p = yn_ddm_summary %>%
  ggplot(aes(day, Mean, color=stim_type))+
  geom_point(position = position_dodge(width=.75))+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), position = position_dodge(width=.75), width=.5)+
  facet_grid(par_name ~ subnum, scale = "free_y")+
  scale_color_brewer(palette = "Dark2")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  labs(x="Day", y = "Parameter Estimate", color="", title = "YN HDDM Estimates (95% HDI) - Hierarchy over days")+
  scale_x_continuous(breaks = seq(1,11,1),
                     labels = c("1", "", "3", "", "5", "", "7", "", "9", "", "11"))

# ggsave(file=paste0(fig_out_path, 'yn_subj_hddm_parests_sub-hierarchy.jpg'), p, height = 6, width=9, units="in")
p

## Compare to old estimates

In [ ]:
stim_types = c("HT", "RE")
subnums = c(601, 609, 611, 619, 621, 629)

in_path = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/hddm/jags_out/old_preNorm'

old_yn_ddm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(subnums)){
    
    cur_stim_type = stim_types[i]
    cur_sub = subnums[j]
    # summary_YN_HDDM_FIT_RE_sub-611.csv
    
    fn = file.path(in_path, paste0('summary_YN_HDDM_FIT_', cur_stim_type,'_sub-', cur_sub,'.csv'))
    
    if(file.exists(fn)){
      cur_summary = read.csv(fn)
      cur_summary = cur_summary %>%
        rename(par = X) %>%
        mutate(stim_type = cur_stim_type,
               subnum = cur_sub)
      
      old_yn_ddm_summary = rbind(old_yn_ddm_summary, cur_summary)
    } else {
      next
    }
  }
}
rm(cur_summary)

In [ ]:
old_yn_ddm_summary

In [ ]:
old_yn_ddm_summary = old_yn_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, subnum, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_group = gsub("p\\[", "", par_group),
         par_group = gsub("\\]", "", par_group)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group != "kappa" & par_group != "pr") %>%
  filter(par_group != "mu") %>% #Not showing group estimates in this plot
  rename(day = par_group) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", ifelse(par_name == "b", "d", ifelse(par_name == "alpha", "sigma", par_name)))) %>%
  mutate(par_name = factor(par_name, levels = c("d", "sigma", "bias", "ndt")),
         day = as.numeric(day))

Plot old vs new mean parameter estimates

In [ ]:
old_yn_ddm_summary %>%
  select(par_name, day, Mean, subnum, stim_type) %>%
  rename(oldEstimate = Mean) %>%
  left_join(yn_ddm_summary %>%
              select(par_name, day, Mean, subnum, stim_type) %>%
              rename(newEstimate = Mean), by = c("par_name", "day", "subnum", "stim_type")) %>%
  filter(par_name == "d") %>%
  ggplot(aes(oldEstimate, newEstimate, color = stim_type))+
  geom_point()+
  facet_grid(par_name ~ subnum)+
  theme(legend.position = "bottom")+
  scale_color_brewer(palette = "Dark2")

In [ ]:
old_yn_ddm_summary %>%
  select(par_name, day, Mean, subnum, stim_type) %>%
  rename(oldEstimate = Mean) %>%
  left_join(yn_ddm_summary %>%
              select(par_name, day, Mean, subnum, stim_type) %>%
              rename(newEstimate = Mean), by = c("par_name", "day", "subnum", "stim_type")) %>%
  filter(par_name == "sigma") %>%
  ggplot(aes(oldEstimate, newEstimate, color = stim_type))+
  geom_point()+
  geom_abline(aes(intercept = 0, slope = 1))+
  facet_grid(par_name ~ subnum)+
  theme(legend.position = "bottom")+
  scale_color_brewer(palette = "Dark2")

In [ ]:
old_yn_ddm_summary %>%
  select(par_name, day, Mean, subnum, stim_type) %>%
  rename(oldEstimate = Mean) %>%
  left_join(yn_ddm_summary %>%
              select(par_name, day, Mean, subnum, stim_type) %>%
              rename(newEstimate = Mean), by = c("par_name", "day", "subnum", "stim_type")) %>%
  filter(par_name == "bias") %>%
  ggplot(aes(oldEstimate, newEstimate, color = stim_type))+
  geom_point()+
  geom_abline(aes(intercept = 0, slope = 1))+
  facet_grid(par_name ~ subnum)+
  theme(legend.position = "bottom")+
  scale_color_brewer(palette = "Dark2")

In [ ]:
old_yn_ddm_summary %>%
  select(par_name, day, Mean, subnum, stim_type) %>%
  rename(oldEstimate = Mean) %>%
  left_join(yn_ddm_summary %>%
              select(par_name, day, Mean, subnum, stim_type) %>%
              rename(newEstimate = Mean), by = c("par_name", "day", "subnum", "stim_type")) %>%
  filter(par_name == "ndt") %>%
  ggplot(aes(oldEstimate, newEstimate, color = stim_type))+
  geom_point()+
  geom_abline(aes(intercept = 0, slope = 1))+
  facet_grid(par_name ~ subnum)+
  theme(legend.position = "bottom")+
  scale_color_brewer(palette = "Dark2")

# Quick checks

Variability of drift rate parameter: `e.p.tau`? Can't tell because it's not monitored.

In [ ]:
cur_summary = read.csv(fn)
unique(cur_summary$X)

Check based on initially confusing observation in previous nb: Is the smaller negative likelihoods for RE compared to HT also the case for these fits?

JAGS output doesn't save likelihood but deviance (-2*log(likelihood)). Deviance is computed for the whole model so in this case for all the days of one subject for each stimulus type (6 subjects x 2 stim types). Below we see that for each subject deviance is lower for the HT stims compared to RE stims (ie. the opposite of what the unnormalized sum nll's would have suggested in the previous nb).

But I'm not sure this is a kosher comparison to evaluate relative model fit.

In [ ]:
stim_types = c("HT", "RE")
subnums = c(601, 609, 611, 619, 621, 629)

in_path = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/hddm/jags_out/old_preNorm'

old_yn_ddm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(subnums)){
    
    cur_stim_type = stim_types[i]
    cur_sub = subnums[j]
    # summary_YN_HDDM_FIT_RE_sub-611.csv
    
    fn = file.path(in_path, paste0('summary_YN_HDDM_FIT_', cur_stim_type,'_sub-', cur_sub,'.csv'))
    
    if(file.exists(fn)){
      cur_summary = read.csv(fn)
      cur_summary = cur_summary %>%
        rename(par = X) %>%
        mutate(stim_type = cur_stim_type,
               subnum = cur_sub)
      
      old_yn_ddm_summary = rbind(old_yn_ddm_summary, cur_summary)
    } else {
      next
    }
  }
}
rm(cur_summary)

In [ ]:
old_yn_ddm_summary %>%
  filter(par == "deviance") %>%
  ggplot(aes(stim_type, Mean, color = as.factor(subnum)))+
  geom_point()+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), width = .1)